In [1]:
from __future__ import annotations

In [2]:
import warnings

warnings.filterwarnings("ignore") 

In [3]:
import sys
sys.path.append("../")

import os
from joblib import dump, load

from C_scrub.d_DASK_CUDA_RAPIDS_data_engineering import scrub_feature_engineering

from collections.abc import Iterator
from functools import partial
from datetime import datetime
import matplotlib.pyplot as plt

import xgboost
from cuml.metrics.accuracy import accuracy_score
import optuna

import dask_cudf
import dask
import dask.dataframe as dd
import distributed
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

# System set-up

In [4]:
BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, 'A_data')
RESOURCES_DIR = os.path.join(BASE_DIR, 'B_resources', 'd_DASK_CUDA_RAPIDS')

In [5]:
dask.config.set({"array.backend": "cupy"})
dask.config.set({"dataframe.backend": "cudf"})

In [6]:
if __name__ == "__main__":
    def get_cluster():
        cluster = LocalCUDACluster(
            device_memory_limit='10GB',
            jit_unspill=True
        )
        client = Client(cluster)
        return client

    client = get_cluster()
    n_workers = len(client.scheduler_info()["workers"])

In [7]:
client

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://127.0.0.1:43543/status,
Dashboard: http://127.0.0.1:43543/status,Workers: 1
Total threads: 1,Total memory: 251.77 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42663,Workers: 1
Dashboard: http://127.0.0.1:43543/status,Total threads: 1
Started: Just now,Total memory: 251.77 GiB
Comm: tcp://127.0.0.1:36879,Total threads: 1
Dashboard: http://127.0.0.1:43593/status,Memory: 251.77 GiB
Nanny: tcp://127.0.0.1:33499,


In [8]:
import time
from contextlib import contextmanager

# Helping time blocks of code
@contextmanager
def timed(txt):
    t0 = time.time()
    yield
    t1 = time.time()
    print("%32s time:  %8.5f" % (txt, t1 - t0))

# Collect data

In [9]:
data_compet = dd.read_csv(os.path.join(DATA_DIR, "train.csv")).set_index('id')
data_orig = dd.read_csv(os.path.join(DATA_DIR, 'horse.csv'))  # Include public data

data_orig['id'] = data_orig.index + data_compet.index.max() + 1
data_orig = data_orig.set_index('id')

data = dd.concat([data_compet, data_orig])
data = data.dropna(subset=['outcome'])
data = data.drop_duplicates()

del data_compet, data_orig

# Prepare data

In [10]:
data = scrub_feature_engineering(data, train=True)
data = dask_cudf.from_cudf(data, npartitions=2)

# Parameters definition

In [11]:
random_seed = 9421

# test split size
split_size = 0.2

#Number of cross-validation folds
n_folds = 4

# number of iteration for RamdomizedSearchCV
XGB_grid_n_iter_search = 100

# Number of parallel optuna jobs to run
N_JOBS = 1

# Number of converging serial trials to run in each job.
N_TRIALS = 10

# The number of training exercises that will be run in total is
# N_JOBS * N_TRIALS * n_folds

# Number of parallel training exercises
N_PARALLEL = 2

# Dask worker instance type and number
N_WORKERS = n_workers

# Split data

In [12]:
train_data, test_data = data.random_split([1- split_size, split_size], shuffle=True, random_state=random_seed)

In [13]:
print(train_data['outcome'].value_counts().compute())

0    618
2    387
1    226
Name: outcome, dtype: int64


In [14]:
print(test_data['outcome'].value_counts().compute())

0    134
2    100
1     69
Name: outcome, dtype: int64


In [15]:
train_data.persist()

if "train_data" in client.datasets:
    client.unpublish_dataset("train_data")
client.publish_dataset(train_data=train_data)

train_data.head()

,rectal_temp,pulse,respiratory_rate,nasogastric_reflux_ph,packed_cell_volume,total_protein,abdomo_protein,numLesions,surgery_0.0,age_0.0,...,lesionCode_1.0,lesionCode_2.0,lesionCode_3.0,lesionCode_5.0,lesionCode_6.0,lesionCode_7.0,lesionCode_8.0,lesionCode_9.0,lesionCode_10.0,outcome
id,,,,,,,,,,,,,,,,,,,,,
591,0.920228,-0.369045,-0.472464,-0.785785,-0.573249,-0.575263,0.386810,0.264269,-0.661571,-0.262659,...,-0.368042,-0.206949,-0.18165,-0.535642,-0.185445,-0.159377,-0.355366,3.060207,-0.2,0
638,-0.257444,-0.545559,-0.568910,0.058562,-0.573249,-0.496918,-0.858120,0.264269,1.511554,-0.262659,...,-0.368042,-0.206949,-0.18165,-0.535642,-0.185445,-0.159377,-0.355366,3.060207,-0.2,1
703,-1.304263,-0.203076,-1.006999,1.465808,-0.096439,-0.377755,1.731854,0.264269,-0.661571,-0.262659,...,-0.368042,-0.206949,-0.18165,1.866917,-0.185445,-0.159377,-0.355366,-0.326775,-0.2,0
583,-2.220231,-0.046462,-0.207439,-1.348683,1.524715,-0.575263,0.257345,0.264269,-0.661571,-0.262659,...,-0.368042,-0.206949,-0.18165,-0.535642,-0.185445,-0.159377,-0.355366,-0.326775,-0.2,2
251,0.004261,0.504303,-0.568910,0.621461,-0.477887,-0.447669,1.302771,0.264269,-0.661571,-0.262659,...,-0.368042,-0.206949,-0.18165,-0.535642,-0.185445,6.274426,-0.355366,-0.326775,-0.2,0


# Train model

In [16]:
# Here we subset data for cross-validation
def make_cv_splits(ddf: dd.DataFrame, n_folds: int = 5) -> Iterator[tuple[dd.DataFrame, dd.DataFrame]]:
    frac = [1 / n_folds] * n_folds
    splits = ddf.random_split(frac, shuffle=True, random_state=random_seed)
    for i in range(n_folds):
        train = [splits[j] for j in range(n_folds) if j != i]
        test = splits[i]
        yield dd.concat(train), test

In [17]:
def train_once(
        train: dd.DataFrame,
        test: dd.DataFrame,
        sem: distributed.Semaphore,
        study_params: dict[str, float],
):
    distributed.secede()
    # Block until there are less than N_PARALLEL train_once critical sections running
    with sem:
        y_train = train["outcome"]
        X_train = train.drop(columns=["outcome"])
        y_test = test["outcome"]
        X_test = test.drop(columns=["outcome"])

        d_train = xgboost.dask.DaskQuantileDMatrix(None, X_train, y_train)
        
        # This has its own internal semaphore with a limit of 1
        model = xgboost.dask.train(
            None,
            {'objective':'multi:softmax',
             'num_class':3,
             "eval_metric": "mlogloss",
             "tree_method": "gpu_hist",
             'predictor':'gpu_predictor',
             **study_params},
            d_train,
            num_boost_round=XGB_grid_n_iter_search,
            evals=[(d_train, "train")],
        )
        predictions = xgboost.dask.predict(None, model, X_test)
        score = accuracy_score(y_test.astype("float32"), predictions, convert_dtype=True)
        return score

In [18]:
def train_model(
        ddf_name: str,
        n_folds: int,
        sem: distributed.Semaphore,
        study_params: dict[str, float],
):
    client = distributed.get_client()
    ddf = client.get_dataset(ddf_name)

    futures = [
        client.submit(train_once, train, test, sem, study_params, pure=False)
        for train, test in make_cv_splits(ddf, n_folds)
    ]

    try:
        distributed.secede()
    except KeyError:
        # Already seceded in a previous iteration of study.optimize()
        pass

    scores = client.gather(futures)
    return sum(scores) / len(scores)

In [19]:
def objective(trial, ddf_name: str, n_folds: int, sem: distributed.Semaphore) -> float:
    study_params = {
        "learning_rate": trial.suggest_float("learning_rate", 0.2, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 1),
        "colsample_bynode": trial.suggest_float("colsample_bynode", 0.3, 1),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.3, 1),
        "reg_lambda": trial.suggest_float("reg_lambda", 0, 1),
        "max_depth": trial.suggest_int("max_depth", 1, 12),
        "max_leaves": trial.suggest_int("max_leaves", 0, 10)
    }
    return train_model(ddf_name, n_folds, sem, study_params)

In [20]:
total_studies = N_JOBS * N_TRIALS * n_folds
print(f"Total studies: {total_studies}")

# Create a single study and run some trials
start = datetime.now()
storage = optuna.integration.DaskStorage()
study = optuna.create_study(storage=storage, study_name="dask_cuml_optuna-HPO", direction="maximize")

# Run N_JOBS in parallel
# each job will run N_TRIALS converging trials in series
# each trial will start n_folds training exercises in parallel
# but only up to N_PARALLEL training exercise will actually submit
# tasks to the scheduler at the same time.

# In pure optuna, we would have used
# study.optimize(objective, n_trials=N_TRIALS, n_jobs=N_JOBS)
sem = distributed.Semaphore(N_PARALLEL)
futures = [
    client.submit(
        study.optimize,
        partial(objective, ddf_name="train_data", n_folds=n_folds, sem=sem),
        n_trials=N_TRIALS,
        pure=False,
    )
    for _ in range(N_JOBS)
]
client.gather(futures)

print(f"Total time:  {datetime.now() - start}")

[I 2023-11-01 13:14:02,626] A new study created in memory with name: dask_cuml_optuna-HPO


Total studies: 40


/home/jgsolar/anaconda3/envs/rapids-23.10/lib/python3.10/site-packages/distributed/protocol/pickle.py:96: ExperimentalWarning: DaskStorage is experimental (supported from v3.1.0). The interface can change in the future.
  return pickle.loads(x)
[13:14:09] task [xgboost.dask-0]:tcp://127.0.0.1:36879 got new rank 0


[0]	train-mlogloss:0.94101
[1]	train-mlogloss:0.81963
[2]	train-mlogloss:0.75351
[3]	train-mlogloss:0.70740
[4]	train-mlogloss:0.66342
[5]	train-mlogloss:0.62739
[6]	train-mlogloss:0.59631
[7]	train-mlogloss:0.57140
[8]	train-mlogloss:0.55041
[9]	train-mlogloss:0.52964
[10]	train-mlogloss:0.51020
[11]	train-mlogloss:0.49055
[12]	train-mlogloss:0.47347
[13]	train-mlogloss:0.46043
[14]	train-mlogloss:0.44601
[15]	train-mlogloss:0.43427
[16]	train-mlogloss:0.42305
[17]	train-mlogloss:0.41186
[18]	train-mlogloss:0.40321
[19]	train-mlogloss:0.39570
[20]	train-mlogloss:0.38530
[21]	train-mlogloss:0.37562
[22]	train-mlogloss:0.36737
[23]	train-mlogloss:0.35941
[24]	train-mlogloss:0.35002
[25]	train-mlogloss:0.34398
[26]	train-mlogloss:0.33723
[27]	train-mlogloss:0.32930
[28]	train-mlogloss:0.32265
[29]	train-mlogloss:0.31641
[30]	train-mlogloss:0.30992
[31]	train-mlogloss:0.30356
[32]	train-mlogloss:0.29954
[33]	train-mlogloss:0.29248
[34]	train-mlogloss:0.28805
[35]	train-mlogloss:0.28250
[3

[13:14:11] task [xgboost.dask-0]:tcp://127.0.0.1:36879 got new rank 0


...

[13:18:00] task [xgboost.dask-0]:tcp://127.0.0.1:36879 got new rank 0


[0]	train-mlogloss:1.01077
[1]	train-mlogloss:0.93537
[2]	train-mlogloss:0.88577
[3]	train-mlogloss:0.85515
[4]	train-mlogloss:0.82642
[5]	train-mlogloss:0.80479
[6]	train-mlogloss:0.78009
[7]	train-mlogloss:0.76238
[8]	train-mlogloss:0.74599
[9]	train-mlogloss:0.72911
[10]	train-mlogloss:0.71563
[11]	train-mlogloss:0.70735
[12]	train-mlogloss:0.69818
[13]	train-mlogloss:0.69073
[14]	train-mlogloss:0.68190
[15]	train-mlogloss:0.67283
[16]	train-mlogloss:0.66590
[17]	train-mlogloss:0.65952
[18]	train-mlogloss:0.65344
[19]	train-mlogloss:0.64722
[20]	train-mlogloss:0.64090
[21]	train-mlogloss:0.63516
[22]	train-mlogloss:0.63017
[23]	train-mlogloss:0.62516
[24]	train-mlogloss:0.62035
[25]	train-mlogloss:0.61599
[26]	train-mlogloss:0.61182
[27]	train-mlogloss:0.60749
[28]	train-mlogloss:0.60410
[29]	train-mlogloss:0.60037
[30]	train-mlogloss:0.59651
[31]	train-mlogloss:0.59306
[32]	train-mlogloss:0.59035
[33]	train-mlogloss:0.58776
[34]	train-mlogloss:0.58454
[35]	train-mlogloss:0.58160
[3

[13:18:02] task [xgboost.dask-0]:tcp://127.0.0.1:36879 got new rank 0


[0]	train-mlogloss:1.00585
[1]	train-mlogloss:0.92439
[2]	train-mlogloss:0.87021
[3]	train-mlogloss:0.83754
[4]	train-mlogloss:0.81030
[5]	train-mlogloss:0.78919
[6]	train-mlogloss:0.76387
[7]	train-mlogloss:0.74568
[8]	train-mlogloss:0.73449
[9]	train-mlogloss:0.71919
[10]	train-mlogloss:0.70937
[11]	train-mlogloss:0.70180
[12]	train-mlogloss:0.69354
[13]	train-mlogloss:0.68527
[14]	train-mlogloss:0.67626
[15]	train-mlogloss:0.66755
[16]	train-mlogloss:0.66060
[17]	train-mlogloss:0.65385
[18]	train-mlogloss:0.64708
[19]	train-mlogloss:0.64217
[20]	train-mlogloss:0.63685
[21]	train-mlogloss:0.63119
[22]	train-mlogloss:0.62678
[23]	train-mlogloss:0.62251
[24]	train-mlogloss:0.61758
[25]	train-mlogloss:0.61384
[26]	train-mlogloss:0.61023
[27]	train-mlogloss:0.60564
[28]	train-mlogloss:0.60226
[29]	train-mlogloss:0.59836
[30]	train-mlogloss:0.59445
[31]	train-mlogloss:0.59079
[32]	train-mlogloss:0.58788
[33]	train-mlogloss:0.58521
[34]	train-mlogloss:0.58231
[35]	train-mlogloss:0.57918
[3

[I 2023-11-01 13:18:07,732] Trial 9 finished with value: 0.7293017655611038 and parameters: {'learning_rate': 0.4657208177648493, 'colsample_bytree': 0.9948705461276774, 'colsample_bynode': 0.9365707895695905, 'colsample_bylevel': 0.3140397974255441, 'reg_lambda': 0.028601447752299336, 'max_depth': 5, 'max_leaves': 2}. Best is trial 4 with value: 0.7438382506370544.


In [21]:
fig = optuna.visualization.matplotlib.plot_optimization_history(study)
fig.legend(loc="lower right")

# Make final tuning

In [22]:
y_train = train_data["outcome"]
X_train = train_data.drop(columns=["outcome"])

d_full_train = xgboost.dask.DaskQuantileDMatrix(None, X_train, y_train)

In [23]:
final_model = xgboost.dask.train(
    None,
    {'objective':'multi:softmax',
     'num_class':3,
     "eval_metric": "mlogloss",
     "tree_method": "gpu_hist",
     'predictor':'gpu_predictor',
     **study.best_params},
    d_full_train,
    num_boost_round=XGB_grid_n_iter_search
)

[13:18:09] task [xgboost.dask-0]:tcp://127.0.0.1:36879 got new rank 0


# Collect final results

In [24]:
y_test = test_data["outcome"]
X_test = test_data.drop(columns=["outcome"])

d_test = xgboost.dask.DaskQuantileDMatrix(None, X_test, y_test)

In [25]:
predictions = xgboost.dask.predict(None, final_model, X_test)
score = accuracy_score(y_test.astype("float32"), predictions, convert_dtype=True)
score

0.735973596572876

In [26]:
dump(final_model['booster'], os.path.join(RESOURCES_DIR, 'model_xgboost_dask_cuml_v0.1.joblib'))

['/home/jgsolar/Documentos/SHARED/Projetos Pessoais/multiplatform_classifier/B_resources/d_DASK_CUDA_RAPIDS/model_xgboost_dask_cuml_v0.1.joblib']

In [27]:
client.shutdown()